In [1]:
from math import log10
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pandas as pd
import os 

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader

from dataset.data_loader_RGB import *
from utils.pytorch_ssim import *

from GAN_model import Generator, Discriminator
from utils.loss import*

torch.manual_seed(1)
device = torch.device("cuda")

In [2]:
# SRGAN parameters

batch_size = 32
num_epochs = 10
lr = 1e-4
threads = 4
upscale_factor = 4

In [3]:
#img_path_low = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-56/train'
#img_path_ref = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-224/train'

img_path_low = '/home/jupyter/dataset/LR_56/train/'
img_path_ref = '/home/jupyter/dataset/HR/train/'

train_set = DatasetSuperRes(img_path_low, img_path_ref)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batch_size, shuffle=True)

In [4]:
netG = Generator(upscale_factor).to(device)
netD = Discriminator().to(device)

#netG.weight_init(mean=0.0, std=0.2)
#netD.weight_init(mean=0.0, std=0.2)

feature_extraction_model = initialize_senet50_2048()

generator_criterion = GeneratorLoss().to(device)
face_loss = FacePerceptionLoss(feature_extraction_model).to(device)
MSE_criterion = nn.MSELoss()

In [5]:
optimizerG = optim.Adam(netG.parameters(), lr=lr)
optimizerD = optim.Adam(netD.parameters(), lr=lr)
    
results = {'d_loss': [], 'g_loss': [], 'd_score': [],
           'g_score': [], 'psnr': [], 'ssim': []}

In [6]:
# Let's pretrain the Generator
pre_train_epochs = 5
for epoch in range(1, pre_train_epochs+1):
    
    netG.train()
    
    for data, target in training_data_loader:
        data, target = data.to(device), target.to(device)
        
        netG.zero_grad()
        loss = MSE_criterion(netG(data), target)
        loss.backward()
        optimizerG.step()
    
    print('Last loss: {}'.format(loss.item()))
    print('Finished pre-training {}/{}'.format(epoch,pre_train_epochs))

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1339: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Last loss: 0.002479344606399536
Finished pre-training 1/5
Last loss: 0.0025044025387614965
Finished pre-training 2/5
Last loss: 0.0025987785775214434
Finished pre-training 3/5
Last loss: 0.002342871855944395
Finished pre-training 4/5
Last loss: 0.0022130270954221487
Finished pre-training 5/5


In [12]:
def train(epoch):
    
    running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 
                       'd_score': 0, 'g_score': 0}
    
    netG.train()
    netD.train()
    iteration = 0
    for data, target in training_data_loader:
        iteration+=1
        g_update_first = True
        batch_size = data.size(0)
        running_results['batch_sizes'] += batch_size

        ############################
        # (1) Update D network: maximize D(x)-1-D(G(z))
        ###########################
        real_img = Variable(target).to(device)
        z = Variable(data).to(device)
        
        fake_img = netG(z)

        netD.zero_grad()
        
        real_out = netD(real_img).mean()
        fake_out = netD(fake_img).mean()
        
        d_loss = 1 - real_out + fake_out
        
        if d_loss.item() > 0.4:
            d_loss.backward(retain_graph=True)
            optimizerD.step()

        ############################
        # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss
        ###########################
        netG.zero_grad()
        
        #mse_adversarial_content_loss = generator_criterion(fake_out, fake_img, real_img)
        #face_perceptual_loss = face_loss(fake_img, real_img)
        
        g_loss = generator_criterion(fake_out, fake_img, real_img)
        
        g_loss.backward()

        fake_img = netG(z)
        fake_out = netD(fake_img).mean()

        optimizerG.step()

        # Loss for current batch before optimization 

        running_results['g_loss'] += g_loss.item() * batch_size
        running_results['d_loss'] += d_loss.item() * batch_size
        running_results['d_score'] += real_out.item() * batch_size
        running_results['g_score'] += fake_out.item() * batch_size

        print("===> Epoch[{}]({}/{}): Discriminator Loss: {:.4f}".format(epoch, iteration, len(training_data_loader), d_loss.item()))

    print('[{}/{}] Loss_D: {:.4f} Loss_G: {:.4f} D(x): {:.4f} D(G(z)): {:.4f}'.format(
            epoch, num_epochs, 
            running_results['d_loss'] / running_results['batch_sizes'],
            running_results['g_loss'] / running_results['batch_sizes'],
            running_results['d_score'] / running_results['batch_sizes'],
            running_results['g_score'] / running_results['batch_sizes']))

    #netG.eval()

    batch_mse = ((fake_img - real_img) ** 2).data.mean()
    batch_ssim = ssim(fake_img, real_img).item()
    batch_psnr = 10 * log10(1 /batch_mse)

    out_path = 'results/'
    out_model_path = 'models/'
    
    if not os.path.exists(out_path):
        os.makedirs(out_path)    
    
    if not os.path.exists(out_model_path):
        os.makedirs(out_model_path)    

    # Save loss\scores\psnr\ssim
    results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
    results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
    results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
    results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
    results['psnr'].append(batch_psnr)
    results['ssim'].append(batch_ssim)

    if epoch % (num_epochs // 2) == 0:

        # Save model parameters
        torch.save(netG, out_model_path + 'SRGAN_x%d_epoch_%d.pth' % (upscale_factor, epoch))
        #torch.save(netD, 'netD_x%d_epoch_%d.pt' % (upscale_factor, epoch))

        data_frame = pd.DataFrame(
            data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
                  'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
            index=range(1, epoch + 1))
        data_frame.to_csv(out_path + 'SRGAN_x' + str(upscale_factor) + '_train_results.csv', index_label='Epoch')

In [13]:
num_epochs = 20

for epoch in range(11, 31):
    train(epoch)

===> Epoch[11](1/562): Discriminator Loss: 0.0467
===> Epoch[11](2/562): Discriminator Loss: 0.0401
===> Epoch[11](3/562): Discriminator Loss: 0.0367
===> Epoch[11](4/562): Discriminator Loss: 0.0321
===> Epoch[11](5/562): Discriminator Loss: 0.0535
===> Epoch[11](6/562): Discriminator Loss: 0.0378
===> Epoch[11](7/562): Discriminator Loss: 0.0427
===> Epoch[11](8/562): Discriminator Loss: 0.0342
===> Epoch[11](9/562): Discriminator Loss: 0.0429
===> Epoch[11](10/562): Discriminator Loss: 0.0294
===> Epoch[11](11/562): Discriminator Loss: 0.0373
===> Epoch[11](12/562): Discriminator Loss: 0.0437
===> Epoch[11](13/562): Discriminator Loss: 0.0338
===> Epoch[11](14/562): Discriminator Loss: 0.0375
===> Epoch[11](15/562): Discriminator Loss: 0.0588
===> Epoch[11](16/562): Discriminator Loss: 0.0633
===> Epoch[11](17/562): Discriminator Loss: 0.0338
===> Epoch[11](18/562): Discriminator Loss: 0.0274
===> Epoch[11](19/562): Discriminator Loss: 0.0429
===> Epoch[11](20/562): Discriminator Lo

===> Epoch[11](161/562): Discriminator Loss: 0.0347
===> Epoch[11](162/562): Discriminator Loss: 0.0251
===> Epoch[11](163/562): Discriminator Loss: 0.0446
===> Epoch[11](164/562): Discriminator Loss: 0.0652
===> Epoch[11](165/562): Discriminator Loss: 0.0703
===> Epoch[11](166/562): Discriminator Loss: 0.0221
===> Epoch[11](167/562): Discriminator Loss: 0.0375
===> Epoch[11](168/562): Discriminator Loss: 0.0381
===> Epoch[11](169/562): Discriminator Loss: 0.0314
===> Epoch[11](170/562): Discriminator Loss: 0.0672
===> Epoch[11](171/562): Discriminator Loss: 0.0720
===> Epoch[11](172/562): Discriminator Loss: 0.0527
===> Epoch[11](173/562): Discriminator Loss: 0.0484
===> Epoch[11](174/562): Discriminator Loss: 0.0330
===> Epoch[11](175/562): Discriminator Loss: 0.0374
===> Epoch[11](176/562): Discriminator Loss: 0.0291
===> Epoch[11](177/562): Discriminator Loss: 0.0317
===> Epoch[11](178/562): Discriminator Loss: 0.0400
===> Epoch[11](179/562): Discriminator Loss: 0.0371
===> Epoch[1

===> Epoch[11](319/562): Discriminator Loss: 0.0291
===> Epoch[11](320/562): Discriminator Loss: 0.0416
===> Epoch[11](321/562): Discriminator Loss: 0.0683
===> Epoch[11](322/562): Discriminator Loss: 0.0575
===> Epoch[11](323/562): Discriminator Loss: 0.0311
===> Epoch[11](324/562): Discriminator Loss: 0.0673
===> Epoch[11](325/562): Discriminator Loss: 0.0577
===> Epoch[11](326/562): Discriminator Loss: 0.0363
===> Epoch[11](327/562): Discriminator Loss: 0.0332
===> Epoch[11](328/562): Discriminator Loss: 0.0342
===> Epoch[11](329/562): Discriminator Loss: 0.0448
===> Epoch[11](330/562): Discriminator Loss: 0.0585
===> Epoch[11](331/562): Discriminator Loss: 0.0413
===> Epoch[11](332/562): Discriminator Loss: 0.0397
===> Epoch[11](333/562): Discriminator Loss: 0.0370
===> Epoch[11](334/562): Discriminator Loss: 0.0341
===> Epoch[11](335/562): Discriminator Loss: 0.0660
===> Epoch[11](336/562): Discriminator Loss: 0.0382
===> Epoch[11](337/562): Discriminator Loss: 0.0476
===> Epoch[1

===> Epoch[11](477/562): Discriminator Loss: 0.0402
===> Epoch[11](478/562): Discriminator Loss: 0.0459
===> Epoch[11](479/562): Discriminator Loss: 0.0333
===> Epoch[11](480/562): Discriminator Loss: 0.0377
===> Epoch[11](481/562): Discriminator Loss: 0.0447
===> Epoch[11](482/562): Discriminator Loss: 0.0468
===> Epoch[11](483/562): Discriminator Loss: 0.0342
===> Epoch[11](484/562): Discriminator Loss: 0.0354
===> Epoch[11](485/562): Discriminator Loss: 0.0351
===> Epoch[11](486/562): Discriminator Loss: 0.0480
===> Epoch[11](487/562): Discriminator Loss: 0.0339
===> Epoch[11](488/562): Discriminator Loss: 0.0334
===> Epoch[11](489/562): Discriminator Loss: 0.0339
===> Epoch[11](490/562): Discriminator Loss: 0.0373
===> Epoch[11](491/562): Discriminator Loss: 0.0261
===> Epoch[11](492/562): Discriminator Loss: 0.0260
===> Epoch[11](493/562): Discriminator Loss: 0.0704
===> Epoch[11](494/562): Discriminator Loss: 0.0342
===> Epoch[11](495/562): Discriminator Loss: 0.0356
===> Epoch[1

===> Epoch[12](73/562): Discriminator Loss: 0.0805
===> Epoch[12](74/562): Discriminator Loss: 0.0402
===> Epoch[12](75/562): Discriminator Loss: 0.0325
===> Epoch[12](76/562): Discriminator Loss: 0.0249
===> Epoch[12](77/562): Discriminator Loss: 0.0309
===> Epoch[12](78/562): Discriminator Loss: 0.0339
===> Epoch[12](79/562): Discriminator Loss: 0.0304
===> Epoch[12](80/562): Discriminator Loss: 0.0402
===> Epoch[12](81/562): Discriminator Loss: 0.0389
===> Epoch[12](82/562): Discriminator Loss: 0.0305
===> Epoch[12](83/562): Discriminator Loss: 0.0374
===> Epoch[12](84/562): Discriminator Loss: 0.0255
===> Epoch[12](85/562): Discriminator Loss: 0.0374
===> Epoch[12](86/562): Discriminator Loss: 0.0278
===> Epoch[12](87/562): Discriminator Loss: 0.0516
===> Epoch[12](88/562): Discriminator Loss: 0.0322
===> Epoch[12](89/562): Discriminator Loss: 0.0440
===> Epoch[12](90/562): Discriminator Loss: 0.0486
===> Epoch[12](91/562): Discriminator Loss: 0.0388
===> Epoch[12](92/562): Discrim

...

===> Epoch[29](489/562): Discriminator Loss: 0.0438
===> Epoch[29](490/562): Discriminator Loss: 0.0877
===> Epoch[29](491/562): Discriminator Loss: 0.0853
===> Epoch[29](492/562): Discriminator Loss: 0.0847
===> Epoch[29](493/562): Discriminator Loss: 0.0811
===> Epoch[29](494/562): Discriminator Loss: 0.0668
===> Epoch[29](495/562): Discriminator Loss: 0.1266
===> Epoch[29](496/562): Discriminator Loss: 0.1063
===> Epoch[29](497/562): Discriminator Loss: 0.0723
===> Epoch[29](498/562): Discriminator Loss: 0.0512
===> Epoch[29](499/562): Discriminator Loss: 0.0531
===> Epoch[29](500/562): Discriminator Loss: 0.0608
===> Epoch[29](501/562): Discriminator Loss: 0.0616
===> Epoch[29](502/562): Discriminator Loss: 0.0566
===> Epoch[29](503/562): Discriminator Loss: 0.0840
===> Epoch[29](504/562): Discriminator Loss: 0.1310
===> Epoch[29](505/562): Discriminator Loss: 0.0367
===> Epoch[29](506/562): Discriminator Loss: 0.1072
===> Epoch[29](507/562): Discriminator Loss: 0.0573
===> Epoch[2

===> Epoch[30](86/562): Discriminator Loss: 0.0751
===> Epoch[30](87/562): Discriminator Loss: 0.0751
===> Epoch[30](88/562): Discriminator Loss: 0.0826
===> Epoch[30](89/562): Discriminator Loss: 0.0756
===> Epoch[30](90/562): Discriminator Loss: 0.0792
===> Epoch[30](91/562): Discriminator Loss: 0.0469
===> Epoch[30](92/562): Discriminator Loss: 0.1013
===> Epoch[30](93/562): Discriminator Loss: 0.0614
===> Epoch[30](94/562): Discriminator Loss: 0.0363
===> Epoch[30](95/562): Discriminator Loss: 0.0804
===> Epoch[30](96/562): Discriminator Loss: 0.0527
===> Epoch[30](97/562): Discriminator Loss: 0.0637
===> Epoch[30](98/562): Discriminator Loss: 0.1266
===> Epoch[30](99/562): Discriminator Loss: 0.0891
===> Epoch[30](100/562): Discriminator Loss: 0.0854
===> Epoch[30](101/562): Discriminator Loss: 0.0263
===> Epoch[30](102/562): Discriminator Loss: 0.0157
===> Epoch[30](103/562): Discriminator Loss: 0.0389
===> Epoch[30](104/562): Discriminator Loss: 0.0749
===> Epoch[30](105/562): D

===> Epoch[30](244/562): Discriminator Loss: 0.1378
===> Epoch[30](245/562): Discriminator Loss: 0.0449
===> Epoch[30](246/562): Discriminator Loss: 0.0456
===> Epoch[30](247/562): Discriminator Loss: 0.0446
===> Epoch[30](248/562): Discriminator Loss: 0.1028
===> Epoch[30](249/562): Discriminator Loss: 0.1329
===> Epoch[30](250/562): Discriminator Loss: 0.0989
===> Epoch[30](251/562): Discriminator Loss: 0.0447
===> Epoch[30](252/562): Discriminator Loss: 0.0573
===> Epoch[30](253/562): Discriminator Loss: 0.0804
===> Epoch[30](254/562): Discriminator Loss: 0.0488
===> Epoch[30](255/562): Discriminator Loss: 0.0713
===> Epoch[30](256/562): Discriminator Loss: 0.0209
===> Epoch[30](257/562): Discriminator Loss: 0.0731
===> Epoch[30](258/562): Discriminator Loss: 0.0219
===> Epoch[30](259/562): Discriminator Loss: 0.0352
===> Epoch[30](260/562): Discriminator Loss: 0.0730
===> Epoch[30](261/562): Discriminator Loss: 0.0503
===> Epoch[30](262/562): Discriminator Loss: 0.0473
===> Epoch[3

===> Epoch[30](402/562): Discriminator Loss: 0.0321
===> Epoch[30](403/562): Discriminator Loss: 0.0164
===> Epoch[30](404/562): Discriminator Loss: 0.0568
===> Epoch[30](405/562): Discriminator Loss: 0.1634
===> Epoch[30](406/562): Discriminator Loss: 0.1112
===> Epoch[30](407/562): Discriminator Loss: 0.0935
===> Epoch[30](408/562): Discriminator Loss: 0.1002
===> Epoch[30](409/562): Discriminator Loss: 0.0891
===> Epoch[30](410/562): Discriminator Loss: 0.1099
===> Epoch[30](411/562): Discriminator Loss: 0.0305
===> Epoch[30](412/562): Discriminator Loss: 0.0939
===> Epoch[30](413/562): Discriminator Loss: 0.0231
===> Epoch[30](414/562): Discriminator Loss: 0.0834
===> Epoch[30](415/562): Discriminator Loss: 0.0186
===> Epoch[30](416/562): Discriminator Loss: 0.1023
===> Epoch[30](417/562): Discriminator Loss: 0.1293
===> Epoch[30](418/562): Discriminator Loss: 0.0776
===> Epoch[30](419/562): Discriminator Loss: 0.0459
===> Epoch[30](420/562): Discriminator Loss: 0.0373
===> Epoch[3

===> Epoch[30](560/562): Discriminator Loss: 0.0319
===> Epoch[30](561/562): Discriminator Loss: 0.0149
===> Epoch[30](562/562): Discriminator Loss: 0.0799
[30/20] Loss_D: 0.0719 Loss_G: 0.0036 D(x): 0.9364 D(G(z)): 0.0083
